In [3]:
# -*- coding: utf-8 -*-
import tensorflow as tf
sess=tf.Session()
#特征数据
features = {
    'department': ['sport', 'sport', 'draw', 'gard', 'trav', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'],
}
#特征列
department = tf.feature_column.categorical_column_with_hash_bucket('department', 10, dtype=tf.string)
print "—————1—————"
print department
print "——————2————"
#组合特征列
columns = [department]
print columns
columns = tf.feature_column.shared_embedding_columns(columns, dimension=10)
#输入层（数据，特征列）
inputs = tf.feature_column.input_layer(features, columns)
#初始化并运行
init = tf.global_variables_initializer()
sess.run(tf.tables_initializer())
sess.run(init)
 
v=sess.run(inputs)
print(v)

—————1—————
HashedCategoricalColumn(key='department', hash_bucket_size=10, dtype=tf.string)
——————2————
[HashedCategoricalColumn(key='department', hash_bucket_size=10, dtype=tf.string)]
[[ 0.38959482 -0.30834228  0.14047201 -0.4914966  -0.3793363   0.16128609
  -0.19833061 -0.31927118 -0.02395553  0.44846094]
 [ 0.38959482 -0.30834228  0.14047201 -0.4914966  -0.3793363   0.16128609
  -0.19833061 -0.31927118 -0.02395553  0.44846094]
 [-0.44056746  0.40502527 -0.27313593  0.0972157  -0.32437634  0.16520736
  -0.20840074  0.03475686  0.27677065 -0.3123621 ]
 [ 0.38959482 -0.30834228  0.14047201 -0.4914966  -0.3793363   0.16128609
  -0.19833061 -0.31927118 -0.02395553  0.44846094]
 [-0.09558015 -0.01415543 -0.38759276 -0.24352945 -0.07161801 -0.39037183
   0.3871828  -0.33830363 -0.13601029 -0.33882076]
 [ 0.15276995 -0.38286322  0.02860998 -0.17966697  0.46105486  0.09873489
   0.07036567  0.11399277  0.05075244 -0.32522264]
 [ 0.36032566  0.1652059  -0.57469493  0.03589788 -0.31461325  0

In [29]:
import tensorflow as tf
from tensorflow.python.estimator.inputs import numpy_io
import numpy as np
import collections
from tensorflow.python.framework import errors
from tensorflow.python.platform import test
from tensorflow.python.training import coordinator
from tensorflow import feature_column

from tensorflow.python.feature_column.feature_column import _LazyBuilder

def test_numeric():

    price = {'price': [[1.], [2.], [3.], [4.]]}  # 4行样本
    builder = _LazyBuilder(price)

    def transform_fn(x):
        return x + 2
    #normalizer_fn: 对该特征下的所有数据进行转换
    price_column = feature_column.numeric_column('price', normalizer_fn=transform_fn)
    #使用_LazyBuilder和inpu_layer来分别进行了测试.效果是一样的.
    price_transformed_tensor = price_column._get_dense_tensor(builder)

    with tf.Session() as session:
        print(session.run([price_transformed_tensor]))

    # 使用input_layer

    price_transformed_tensor = feature_column.input_layer(price, [price_column])

    with tf.Session() as session:
        print('use input_layer' + '_' * 40)
        print(session.run([price_transformed_tensor]))
        
#test_numeric()

def test_bucketized_column():

    price = {'price': [[5.], [15.], [25.], [35.]]}  # 4行样本

    price_column = feature_column.numeric_column('price')
    #source_column: 必须是numeric_column
    #boundaries: 不同的桶。boundaries=[0., 1., 2.],产生的bucket就是, (-inf, 0.), [0., 1.), [1., 2.), and [2., +inf), 每
    #一个区间分别表示0, 1, 2, 3,所以相当于分桶分了4个.
    bucket_price = feature_column.bucketized_column(price_column, [0, 10, 20, 30, 40])

    price_bucket_tensor = feature_column.input_layer(price, [bucket_price])

    with tf.Session() as session:
        print(session.run([price_bucket_tensor]))
        
#我们看到分桶之后，会直接转换成one-hot形式的。
#test_bucketized_column()

def test_categorical_column_with_vocabulary_list():

    color_data = {'color': [['R', 'R'], ['G', 'R'], ['B', 'G'], ['A', 'A']]}  # 4行样本

    builder = _LazyBuilder(color_data)
    #categorical_column_with_vocabulary_list来说返回的是sparser_tensor
    #注意 id_tensor 这个是有效的，另外一个是None. 对于线性模型来说是可以直接使用sparser_tensor的
    color_column = feature_column.categorical_column_with_vocabulary_list(
        'color', ['R', 'G', 'B'], dtype=tf.string, default_value=-1
    )

    color_column_tensor = color_column._get_sparse_tensors(builder)

    with tf.Session() as session:
        session.run(tf.global_variables_initializer())

        session.run(tf.tables_initializer())

        print(session.run([color_column_tensor.id_tensor]))
    #对于深度模型来说，需要将sparser转换成dense，所以也就有了indicator_column 这个函数的出现。
    # 将稀疏的转换成dense，也就是one-hot形式，只是multi-hot
    #indicator_column的作用就是将category产生的sparser tensor转换成dense tensor.
    color_column_identy = feature_column.indicator_column(color_column)
    #input_layer: 只接受dense tensor
    color_dense_tensor = feature_column.input_layer(color_data, [color_column_identy])

    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        #tables_initializer: 在sparser的时候使用的，如果不进行初始化会出现 Table not initialized.
        session.run(tf.tables_initializer())

        print('use input_layer' + '_' * 40)
        print(session.run([color_dense_tensor]))
        
#test_categorical_column_with_vocabulary_list()

def test_categorical_column_with_hash_bucket():

    color_data = {'color': [['R'], ['G'], ['B'], ['A']]}  # 4行样本
    builder = _LazyBuilder(color_data)
    #当category的数量很多，也就无法使用指定category的方法来处理了，那么，可以使用这种哈希分桶的方式来进行处理
    #毕竟对于hash_bucket来说，对于bucket_size的选取是个问题
    color_column = feature_column.categorical_column_with_hash_bucket('color', 7)
    color_column_tensor = color_column._get_sparse_tensors(builder)

    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        session.run(tf.tables_initializer())
        print(session.run([color_column_tensor.id_tensor]))

    # 将稀疏的转换成dense，也就是one-hot形式，只是multi-hot
    color_column_identy = feature_column.indicator_column(color_column)
    color_dense_tensor = feature_column.input_layer(color_data, [color_column_identy])

    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        session.run(tf.tables_initializer())
        print('use input_layer' + '_' * 40)
        print(session.run([color_dense_tensor]))
#从上面看这种hash分桶的方法，在hash_size的选择上是很重要的
#现在选择3，对于R 和 B 来说分桶到一个烈面了；对于 G和A 分桶到一个里面了
#当将 hash_size=7来测试, R G B A就都分到了不同的桶中，所以值越大也容易精确的分桶.

#test_categorical_column_with_hash_bucket()

def test_embedding():
    color_data = {'color': [['R'], ['G'], ['B'], ['A']]}  # 4行样本

    color_column = feature_column.categorical_column_with_vocabulary_list(
        'color', ['R', 'G', 'B'], dtype=tf.string, default_value=-1
    )
    #每一个都会转换成5个维度的数据，并且使用高斯分布来进行初始化。
    #因为A 没有在catergorical_column中出现，所以使用了0进行初始化.
    color_embeding = feature_column.embedding_column(color_column, 5)
    color_embeding_dense_tensor = feature_column.input_layer(color_data, [color_embeding])

    with tf.Session() as session:
        session.run(tf.global_variables_initializer())

        session.run(tf.tables_initializer())

        print('embeding' + '_' * 40)
        print(session.run([color_embeding_dense_tensor]))
        
#test_embedding()

def test_weighted_categorical_column():
    color_data = {'color': [['R'], ['G'], ['B'], ['A']],
                  'weight': [[1.0], [2.0], [4.0], [8.0]]}  # 4行样本

    color_column = feature_column.categorical_column_with_vocabulary_list(
        'color', ['R', 'G', 'B'], dtype=tf.string, default_value=-1
    )

    color_weight_categorical_column = feature_column.weighted_categorical_column(color_column, 'weight')

    builder = _LazyBuilder(color_data)

    with tf.Session() as session:
        id_tensor, weight = color_weight_categorical_column._get_sparse_tensors(builder)

        session.run(tf.global_variables_initializer())

        session.run(tf.tables_initializer())

        print('weighted categorical' + '-' * 40)

        print(session.run([id_tensor]))
        print('-' * 40)
        print(session.run([weight]))

#test_weighted_categorical_column()

#对所有特征进行线性加权操作
def get_linear_model_bias():
    with tf.variable_scope('linear_model', reuse=True):
        return tf.get_variable('bias_weights')
def get_linear_model_column_var(column):
    return tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                             'linear_model/' + column.name)[0]
def test_linear_model():
    """
    测试线性模型
    :return:
    """

    featrues = {
        'price': [[1.0], [5.0], [10.0]],
        'color': [['R'], ['G'], ['B']]
    }
    price_column = feature_column.numeric_column('price')
    color_column = feature_column.categorical_column_with_vocabulary_list('color', ['R', 'G', 'B'])
    prediction = feature_column.linear_model(featrues, [price_column, color_column])

    bias = get_linear_model_bias()
    price_var = get_linear_model_column_var(price_column)
    color_var = get_linear_model_column_var(color_column)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        sess.run(tf.tables_initializer())

        sess.run(bias.assign([7.0]))
        sess.run(price_var.assign([[10.0]]))
        sess.run(color_var.assign([[2.0], [2.0], [2.0]]))

        predication_result = sess.run([prediction])

        print(predication_result)

#test_linear_model()

def test_crossed_column():
    """
    crossed column测试
    :return:
    """
    featrues = {
        'price': [['A', 'A'], ['B', 'D'], ['C', 'A']],
        'color': [['R', 'R'], ['G', 'G'], ['B', 'B']]
    }

    price = feature_column.categorical_column_with_vocabulary_list('price',
                                                                   ['A', 'B', 'C', 'D'])
    color = feature_column.categorical_column_with_vocabulary_list('color',
                                                                   ['R', 'G', 'B'])
    p_x_c = feature_column.crossed_column([price, color], 16)

    p_x_c_identy = feature_column.indicator_column(p_x_c)

    p_x_c_identy_dense_tensor = feature_column.input_layer(featrues, [p_x_c_identy])

    with tf.Session() as session:
        session.run(tf.global_variables_initializer())

        session.run(tf.tables_initializer())

        print('use input_layer' + '_' * 40)
        print(session.run([p_x_c_identy_dense_tensor]))

test_crossed_column()

use input_layer________________________________________
[array([[0., 0., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 2., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 2., 0.]],
      dtype=float32)]
